In [1]:
import os
import numpy as np
import opensim as osim

In [2]:
path_model = './model/scaled_lenhart_model.osim'
model = osim.Model(path_model)

[info] Loaded model scale_tool from file ./model/scaled_lenhart_model.osim


In [3]:
ellipsoid = osim.WrapEllipsoid()
ellipsoid.set_xyz_body_rotation(osim.Vec3([0, 0, -0.28]))
ellipsoid.set_translation(osim.Vec3([-0.04, -0.353, 0]))
ellipsoid.set_quadrant('all')
ellipsoid.set_dimensions(osim.Vec3([0.08, 0.035, 0.2]))
ellipsoid.setName("KnExt_at_fem_r_2")

In [5]:
body_add_ellipsoid = 'femur_r'

bodyset = model.getBodySet()

for idx in range(bodyset.getSize()):
    body = bodyset.get(idx)
    name = body.getName()
    if name == body_add_ellipsoid:
        print(name)
        body.addWrapObject(ellipsoid)
        print(f'Added wrap')

femur_r
Added wrap


In [6]:
def update_path_point(muscle, pt_idx, indices, locations):
    if type(indices) not in [list, tuple]:
        indices = [indices,]
    if type(locations) not in [list, tuple]:
        locations = [locations,]
    geopath = muscle.getGeometryPath()
    path_pointset = geopath.getPathPointSet()
    pt = path_pointset.get(pt_idx)
    orig_loc = [pt.getLocation(state)[x] for x in range(3)]
    pt_ = [pt.getLocation(state)[x] for x in range(3)]
    for idx, new_loc in zip(indices, locations):
        pt_[idx] = new_loc
        
    path_point_ = osim.PathPoint.safeDownCast(pt)
    path_point_.setLocation(osim.Vec3(pt_))
    updated_loc = [geopath.getPathPointSet().get(pt_idx).getLocation(state)[x] for x in range(3)]
    print(f'Orig location: {orig_loc}\nNew location: {updated_loc}')

    

In [7]:
quads = [
    'recfem_r',
    'vasint_r',
    'vaslat_r',
    'vasmed_r'
]

muscles = model.getMuscles()
n_muscles = muscles.getSize()

state = model.initSystem()

for muscle_idx in range(n_muscles):
    muscle = muscles.get(muscle_idx)
    name = muscle.getName()
    if name in quads:
        # Update each quads muscle to wrap around new object. 
        geopath = muscle.getGeometryPath()
        wrapset = geopath.getWrapSet()
        wrap = wrapset.get(0)
        wrap.set_wrap_object(0, ellipsoid.getName())
    
    if name == 'recfem_r':
        print(muscle_idx, name)
        update_path_point(
            muscle=muscle, 
            pt_idx=1, 
            indices=[0,], 
            locations=[0.0,]
        )
    elif name == 'vasint_r':
        print(muscle_idx, name)
        update_path_point(
            muscle=muscle, 
            pt_idx=2, 
            indices=[0, 1], 
            locations=[0.0, 0.0156828]
        )
    elif name == 'vasmed_r':
        print(muscle_idx, name)
        update_path_point(muscle=muscle, pt_idx=2, indices=[0], locations=[0.0])

33 recfem_r
Orig location: [-0.000102437, 0.0149698, 0.0011381]
New location: [0.0, 0.0149698, 0.0011381]
41 vasint_r
Orig location: [-0.00380437, 0.0180608, -0.00035737]
New location: [0.0, 0.0156828, -0.00035737]
43 vasmed_r
Orig location: [-0.000394407, 0.0156828, -0.0093351]
New location: [0.0, 0.0156828, -0.0093351]


In [8]:
model.printToXML('./model/scaled_lenhart_model_updated_wrap.osim')

True